In [1]:
! pip install torch torchvision opencv-python pillow
! pip install ultralytics
! pip install nuscenes-devkit &> /dev/null  # Install nuScenes.
! pip install pyquaternion
! pip install pyntcloud
! pip install filterpy

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [2]:
from ultralytics import YOLO
from IPython.display import display, Image
from IPython import display
import os
import shutil
from tqdm import tqdm
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import Box
from nuscenes.utils.geometry_utils import view_points
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
from pyquaternion import Quaternion
from nuscenes.utils.data_classes import LidarPointCloud, RadarPointCloud
from nuscenes.utils.geometry_utils import transform_matrix  # Importing the transform_matrix function
import json
import pyntcloud
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import RadarPointCloud
from nuscenes.utils.geometry_utils import view_points, transform_matrix
from filterpy.kalman import KalmanFilter
from filterpy.kalman import ExtendedKalmanFilter as EKF
from scipy.spatial.distance import mahalanobis

/usr/local/lib/python3.10/dist-packages/pyntcloud/utils/numba.py:5: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def groupby_count(xyz, indices, out):
/usr/local/lib/python3.10/dist-packages/pyntcloud/utils/numba.py:12: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def groupby_sum(xyz, indices, N, out):
/usr/local/lib/python3.10/dist-packages/pyntcloud/utils/nu

In [3]:
model = YOLO('/content/drive/MyDrive/best.pt')

In [ ]:
metrics = model.val(data = '/content/drive/MyDrive/TestdataDetection/data.yaml')

Ultralytics YOLOv8.2.58 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)


val: Scanning /content/drive/MyDrive/TestdataDetection/labels/val... 41 images, 0 backgrounds, 0 corrupt: 100%|██████████| 41/41 [00:22<00:00,  1.85it/s]

val: New cache created: /content/drive/MyDrive/TestdataDetection/labels/val.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:08<00:00,  2.78s/it]


                   all         41        365      0.612      0.426      0.526      0.257
        movable_object         18         45      0.591      0.267      0.388      0.161
               vehicle         41        320      0.633      0.584      0.663      0.352
Speed: 16.6ms preprocess, 52.1ms inference, 0.0ms loss, 5.6ms postprocess per image
Results saved to runs/detect/val3


In [4]:
pred = model('/content/drive/MyDrive/testdata3/images/n015-2018-11-21-19-38-26+0800__CAM_FRONT__1542800367912460.jpg')


image 1/1 /content/drive/MyDrive/testdata3/images/n015-2018-11-21-19-38-26+0800__CAM_FRONT__1542800367912460.jpg: 736x1280 3 vehicles, 59.5ms
Speed: 29.8ms preprocess, 59.5ms inference, 814.8ms postprocess per image at shape (1, 3, 736, 1280)


In [5]:
pred[0].save('prediction.jpg')

'prediction.jpg'

In [6]:
pred[0].boxes.data

tensor([[3.3287e-01, 4.8623e+02, 2.7143e+02, 6.9103e+02, 9.2746e-01, 4.0000e+00],
        [4.1847e+02, 4.9120e+02, 6.2375e+02, 5.7056e+02, 7.0229e-01, 4.0000e+00],
        [3.0177e+02, 4.8964e+02, 4.4558e+02, 5.4219e+02, 7.0115e-01, 4.0000e+00]], device='cuda:0')

In [8]:
def initialize_ekf(x, y, vx, vy, w, h):
    dt = 0.5  # Time step
    ekf = EKF(dim_x=6, dim_z=4)  # 6 state variables, 4 measurements

    # State vector: [x, y, vx, vy, w, h]
    ekf.x = np.array([x, y, vx, vy, w, h])

    # State transition matrix
    ekf.F = np.array([[1, 0, dt, 0,  0, 0],
                      [0, 1, 0, dt,  0, 0],
                      [0, 0, 1,  0,  0, 0],
                      [0, 0, 0,  1,  0, 0],
                      [0, 0, 0,  0,  1, 0],
                      [0, 0, 0,  0,  0, 1]])

    # Measurement function matrix
    ekf.H = np.array([[1, 0, 0, 0, 0, 0],
                      [0, 1, 0, 0, 0, 0],
                      [0, 1, 1, 0, 0, 0],
                      [0, 1, 0, 1, 0, 0],
                      [0, 0, 0, 0, 1, 0],
                      [0, 0, 0, 0, 0, 1]])

    # Process noise covariance
    ekf.Q = np.eye(6) * 0.1  # Adjust for your process noise

    # Measurement noise covariance
    ekf.R = np.eye(4) * 1.0  # Adjust for your measurement noise

    # Initial state covariance
    ekf.P *= 1.0  # Adjust initial uncertainty

    return ekf

In [9]:
from scipy.optimize import linear_sum_assignment

def euclidean_distance(detection, tracker):
    """
    Calculate the Euclidean distance between the detection and the tracker.
    Parameters:
    detection (np.ndarray): Detection coordinates [x, y]
    tracker (np.ndarray): Tracker coordinates [x, y]
    Returns:
    float: Euclidean distance
    """
    return np.sqrt((detection[0] - tracker[0]) ** 2 + (detection[1] - tracker[1]) ** 2)

def associate_detections_to_trackers_hungarian(detections, trackers, max_distance):
    """
    Assign detections to tracked objects using the Hungarian algorithm.
    Parameters:
    detections (np.ndarray): Array of detections in the format [[x, y], ...]
    trackers (np.ndarray): Array of trackers in the format [[x, y], ...]
    max_distance (float): Maximum distance to associate detection to tracker
    Returns:
    matches (np.ndarray): Array of matched indices [(detection_idx, tracker_idx), ...]
    unmatched_detections (np.ndarray): Array of unmatched detection indices
    unmatched_trackers (np.ndarray): Array of unmatched tracker indices
    """
    if len(trackers) == 0:
        return np.empty((0, 2), dtype=int), np.arange(len(detections)), np.empty((0, 2), dtype=int)

    distance_matrix = np.zeros((len(detections), len(trackers)), dtype=np.float32)

    for d, det in enumerate(detections):
        for t, trk in enumerate(trackers):
            distance_matrix[d, t] = euclidean_distance(det, trk)

    row_indices, col_indices = linear_sum_assignment(distance_matrix)

    matches = []
    unmatched_detections = []
    unmatched_trackers = []

    for d, det in enumerate(detections):
        if d not in row_indices:
            unmatched_detections.append(d)

    for t, trk in enumerate(trackers):
        if t not in col_indices:
            unmatched_trackers.append(t)

    for row, col in zip(row_indices, col_indices):
        if distance_matrix[row, col] < max_distance:
            matches.append((row, col))
        else:
            unmatched_detections.append(row)
            unmatched_trackers.append(col)

    matches = np.array(matches)
    unmatched_detections = np.array(unmatched_detections)
    unmatched_trackers = np.array(unmatched_trackers)

    return matches, unmatched_detections, unmatched_trackers


In [10]:
object_id_counter = 0
object_trackers = {}

def initialize_trackers(detections):
    global object_id_counter
    global object_trackers # Declare object_trackers as global
    for detection in detections:
        x, y, vx, vy, w, h = detection
        ekf = initialize_ekf(x, y, vx, vy, w, h)
        object_trackers[object_id_counter] = {'ekf': ekf, 'bbox': [x, y], 'velocity': [vx, vy], "width" : w, "height": h}
        object_id_counter += 1

def measurement_function(state):
    return np.array([state[0], state[1]])  # Only x and y

def measurement_jacobian(state):
    return np.array([[1, 0, 0, 0, 0, 0],  # ∂x/∂x
                     [0, 1, 0, 0, 0, 0]])  # ∂y/∂y


def predict_and_associate(detections, max_distance=50):
    global object_id_counter
    global object_trackers # Declare object_trackers as global
    for obj_id, tracker in object_trackers.items():
        tracker['ekf'].predict()
        tracker['updated'] = False

    # Prepare detections and trackers for Hungarian association
    detections_positions = np.array([[x, y] for x, y, _, _ , ___, ______ in detections])
    trackers_positions = np.array([[tracker['bbox'][0], tracker['bbox'][1]] for tracker in object_trackers.values()])

    # Associate detections to trackers using Hungarian algorithm
    matches, unmatched_detections, unmatched_trackers = associate_detections_to_trackers_hungarian(detections_positions, trackers_positions, max_distance)
    print(matches)
    print(unmatched_detections)
    print(unmatched_trackers)


    # Update matched trackers
    for detection_idx, tracker_idx in matches:
        detection = detections[detection_idx]
        tracker_id = list(object_trackers.keys())[tracker_idx]
        x, y, vx, vy, w, h = detection
        # Calculate HJacobian and Hx using the current state of the tracker
        predicted_state = object_trackers[tracker_id]['ekf'].x_post  # Get the predicted state
        HJacobian = measurement_jacobian(predicted_state)
        Hx = measurement_function(predicted_state)
        R = np.eye(2) * 0.1  # Adjust the value 0.1 based on your sensor noise characteristics
        object_trackers[tracker_id]['ekf'].update(np.array([x, y]),measurement_jacobian,measurement_function, R = R)
        object_trackers[tracker_id]['bbox'] = [x, y]  # Update bounding box
        object_trackers[tracker_id]['velocity'] = [vx, vy]
        object_trackers[tracker_id]['width'] = w
        object_trackers[tracker_id]['height'] = h
        object_trackers[tracker_id]['updated'] = True

    # Initialize new trackers for unmatched detections
    for detection_idx in unmatched_detections:
        x, y, vx, vy, w, h = detections[detection_idx]
        new_ekf = initialize_ekf(x, y, vx, vy, w, h)
        object_trackers[object_id_counter] = {'ekf': new_ekf, 'bbox': [x, y], 'velocity': [vx, vy], "width": w, "height": h, 'updated': True}
        object_id_counter += 1

    # Remove trackers that were not updated
    object_trackers = {obj_id: tracker for obj_id, tracker in object_trackers.items() if tracker['updated']}

In [11]:
#function to project radar points onto image plane
def radar_to_camera_projection(camera_trans, camera_rot, cam_intrinsic, radar_trans, radar_rot,radar_points):

    #create tranform for radar to ego transform and tranform the objects from radar to ego frame
    radar_to_ego = transform_matrix(radar_trans, Quaternion(radar_rot), inverse=False)
    radar_point_ego = np.dot(radar_to_ego, np.vstack((radar_points, np.ones(radar_points.shape[1]))))

    #create tranform for ego to camera transform and tranform the objects from ego to camera frame
    ego_to_cam = transform_matrix(camera_trans, Quaternion(camera_rot), inverse=True)
    radar_points_camera = np.dot(ego_to_cam, radar_point_ego)

    # Project radar points onto the image plane
    radar_points_2d = view_points(radar_points_camera[:3, :], cam_intrinsic, normalize=True)

    return radar_points_2d

In [12]:
#function for velocity association with bboxes
def associate_radar_points_to_boxes(radar_points, cam_detections):
    associated_velocities = []
    for detection in cam_detections:
        #print(detection)
        #print(radar_points)
        center_x,center_y,min_x,min_y,max_x,max_y, width, height = detection[0],detection[1],detection[2],detection[3],detection[4],detection[5],detection[6],detection[7]
        #print(center_x,center_y,min_x,min_y,max_x,max_y)
        box_center = np.array([center_x, center_y])

        object_points = []
        for x, y, vx, vy in radar_points:
            radar_point = np.array([x, y])
            if min_x <= x <= max_x and min_y <= y <= max_y:
              object_points.append([x,y,vx,vy])


        if len(object_points) > 0:
            object_points = np.array(object_points)
            velocities = object_points[:,2:4]  # Assuming vx, vy are in rows 3 and 4
            mean_velocity = np.mean(velocities, axis=0)
            #print(mean_velocity)
            associated_velocities.append([center_x, center_y,mean_velocity[0][0].tolist(),mean_velocity[1][0].tolist(), width, height])
            #print(mean_velocity)
        else:
            mean_velocity = np.array([0.0, 0.0])  # Default velocity for no points
            #print(mean_velocity)
            associated_velocities.append([center_x, center_y,mean_velocity[0].tolist(),mean_velocity[1].tolist(), width, height])
    return associated_velocities

In [7]:
def calculate_the_center_dims(detections):
    # Initialize empty list to store detections with corners
    detections_with_corners = []
    for detection in detections:
        center_x = (detection[0] + detection[2]) / 2
        center_y = (detection[1] + detection[3]) / 2
        width = detection[2] - detection[0]
        height = detection[3] - detection[1]
        min_x = detection[0]
        min_y = detection[1]
        max_x = detection[2]
        max_y = detection[3]
        detections_with_corners.append([center_x, center_y, min_x, min_y, max_x, max_y, width, height])
    return detections_with_corners

In [18]:
#open the Json file with test data
with open('/content/drive/MyDrive/testdata2/ground_truth_reindexed.json') as file:
  data = json.load(file)

In [14]:
def process_data(data):
    # Initialize empty list to store predictions
    predictions = []
    for frame in tqdm(data):
      pred_data = {"frame": [], "Image_path":[], "Radar_path":[],  "objects": []}
      frame_id = frame['frame']
      image_path = frame["Image_path"]
      radar_path = frame["Radar_path"]
      pred_data["frame"] = frame_id
      pred_data["Image_path"] = image_path
      pred_data["Radar_path"] = radar_path

      #Loading radar pointcloud
      radar_pcd = RadarPointCloud.from_file(frame["Radar_path"])
      radar_points = radar_pcd.points[:3, :]  # Take only x, y, z coordinates

      #extracting sensor extrinsics and intrinsics from the json frame
      camera_translation = frame["camera_translation"] #the spelling mistake in json file
      camera_rotation = frame["camera_rotation"]
      camera_intrinsic = frame["camera_intrinsic"]
      radar_translation = frame["radar_translation"]
      radar_rotation = frame["radar_rotation"]
      camera_intrinsic = np.array(camera_intrinsic)

      #Projecting radarpoints onto the imageplane
      radar_points_2d = radar_to_camera_projection(camera_translation, camera_rotation, camera_intrinsic, radar_translation, radar_rotation,radar_points)

      #creating radar list with location and velocity
      radarpoints = []
      radarpoints.append(np.vstack((radar_points_2d[:2, :], radar_pcd.points[6:8, :]))) #radar pointcloud contains vx an vy on column 6 and 7
      radarpointsarray = np.array(radarpoints)
      radarpointsarray = radarpointsarray.T

      #Projecting radarpoints onto the imageplane
      radar_points_2d = radar_to_camera_projection(camera_translation, camera_rotation, camera_intrinsic, radar_translation, radar_rotation,radar_points)

      #creating radar list with location and velocity
      radarpoints = []
      radarpoints.append(np.vstack((radar_points_2d[:2, :], radar_pcd.points[6:8, :]))) #radar pointcloud contains vx an vy on column 6 and 7
      radarpointsarray = np.array(radarpoints)
      radarpointsarray = radarpointsarray.T
      radarpointsarray

      #using YOLO to get camera detections
      results = model.predict(image_path)
      #creating a list of bboxes
      bbox = results[0].boxes.data

      #converting the tensor to a list
      detections = bbox.tolist()

      #calculating the corner of the bboxes
      detcorners = calculate_the_center_dims(detections)

      #associating the boxes with the radar velocity
      bboxes_with_velocity = associate_radar_points_to_boxes(radarpointsarray,detcorners)

      if len(predictions) == 0:
         print("Initializing trackers")
         initialize_trackers(detections=bboxes_with_velocity)
      else:
          print("Predicting and associating")
          predict_and_associate(detections=bboxes_with_velocity)

      for obj_id, tracker in object_trackers.items():
          bbox = tracker['bbox']
          velocity = tracker['velocity']
          width = tracker['width']
          height = tracker['height']
          pred_data["objects"].append({"id": obj_id, "bbox": bbox, "velocity": velocity, "width": width, "height": height})

      predictions.append(pred_data)

    with open('predictions.json', 'w') as f:
        json.dump(predictions, f)


In [19]:
process_data(data)

  0%|          | 0/41 [00:00<?, ?it/s]


image 1/1 /content/drive/MyDrive/testdata2/images/n015-2018-10-08-15-36-50+0800__CAM_FRONT__1538984233512470.jpg: 736x1280 6 humans, 12.7ms
Speed: 9.2ms preprocess, 12.7ms inference, 2.1ms postprocess per image at shape (1, 3, 736, 1280)


  2%|▏         | 1/41 [00:01<01:17,  1.94s/it]

Initializing trackers

image 1/1 /content/drive/MyDrive/testdata2/images/n015-2018-10-08-15-36-50+0800__CAM_FRONT__1538984234012472.jpg: 736x1280 12 humans, 1 vehicle, 12.6ms
Speed: 6.4ms preprocess, 12.6ms inference, 1.3ms postprocess per image at shape (1, 3, 736, 1280)


  5%|▍         | 2/41 [00:03<01:14,  1.92s/it]

...

Predicting and associating
[]
[0 1]
[0 1]

image 1/1 /content/drive/MyDrive/testdata2/images/n015-2018-10-08-15-36-50+0800__CAM_FRONT__1538984252912460.jpg: 736x1280 1 vehicle, 12.5ms
Speed: 6.4ms preprocess, 12.5ms inference, 1.4ms postprocess per image at shape (1, 3, 736, 1280)


 98%|█████████▊| 40/41 [01:10<00:01,  1.69s/it]

Predicting and associating
[]
[0]
[1 0]

image 1/1 /content/drive/MyDrive/testdata2/images/n015-2018-10-08-15-36-50+0800__CAM_FRONT__1538984253412460.jpg: 736x1280 (no detections), 12.5ms
Speed: 5.9ms preprocess, 12.5ms inference, 0.6ms postprocess per image at shape (1, 3, 736, 1280)


100%|██████████| 41/41 [01:12<00:00,  1.76s/it]

Predicting and associating
[]
[]
[0]


In [16]:
#formatting Json file to make it readable
def format_json_file(input_file_path, output_file_path=None):
    # Read the JSON data from the input file
    with open(input_file_path, 'r') as input_file:
        data = json.load(input_file)

    # Format the JSON data
    formatted_json = json.dumps(data, indent=4)

    # Print formatted JSON data
    print(formatted_json)

    # Write the formatted JSON data to the output file (if provided)
    if output_file_path:
        with open(output_file_path, 'w') as output_file:
            output_file.write(formatted_json)

In [21]:
input_file_path = '/content/predictions.json'  # Path to your input JSON file
output_file_path = '/content/drive/MyDrive/testdata2/predictions_latest.json'  # Path to your output JSON file (optional)
format_json_file(input_file_path, output_file_path)

[
    {
        "frame": "b5989651183643369174912bc5641d3b",
        "Image_path": "/content/drive/MyDrive/testdata2/images/n015-2018-10-08-15-36-50+0800__CAM_FRONT__1538984233512470.jpg",
        "Radar_path": "/content/drive/MyDrive/testdata2/radar/n015-2018-10-08-15-36-50+0800__RADAR_FRONT__1538984233560834.pcd",
        "objects": [
            {
                "id": 148,
                "bbox": [
                    992.3377990722656,
                    503.0427551269531
                ],
                "velocity": [
                    0.0,
                    0.0
                ],
                "width": 31.47125244140625,
                "height": 19.83978271484375
            },
            {
                "id": 150,
                "bbox": [
                    844.9976501464844,
                    536.3502349853516
                ],
                "velocity": [
                    -9.916666666666666,
                    1.1666666666666667
                ],
      